# Libraries

In [2]:
from ipywidgets import (
    Layout,
    Label, Button, HTML,
    IntSlider,
    DatePicker,
    Dropdown,
    SelectMultiple, SelectionSlider,
    HBox, VBox, Output,
    IntProgress,
    FileUpload
)

from ipyleaflet import (
    Map,
    basemap_to_tiles, basemaps, TileLayer, MagnifyingGlass,
    ZoomControl, LayersControl, DrawControl, WidgetControl, SearchControl, FullScreenControl, ScaleControl,
    Popup, GeoJSON
)

import pystac_client
import planetary_computer

import threading
import json
from datetime import datetime, timedelta
import mercantile
import requests
import geojson

from shapely.geometry import shape
from shapely.affinity import scale

# Util

In [6]:
class UtilScenes(object):
    colorMessage = lambda msg, color: f"<b style='color:{color};'> {msg} </b>"

    @staticmethod
    def search(collections, date_ini, date_end, geom_json):
        """
        Return: { 'scenes' = { scene: Item collection } , 'htmlMessage'}
        """
        def searchScenes(time_of_interest):
            catalog = pystac_client.Client.open(
                "https://planetarycomputer.microsoft.com/api/stac/v1",
                modifier=planetary_computer.sign_inplace,
            )
            search = catalog.search(
                collections=collections,
                intersects=geom_json,
                datetime=time_of_interest,
                # query={"eo:cloud_cover": {"lt": 10}},
            )
            return { item.id: item for item in search.item_collection() }

        v_return = { 'scenes': None,  'htmlMessage': None }
        formatDate = '%Y-%m-%d'
        strdate_ini, strdate_end = date_ini.strftime( formatDate ), date_end.strftime( formatDate )
        if date_ini > date_end:
            v_return['htmlMessage'] = f"Initial Date ({__class__.colorMessage(  strdate_ini, 'red' )}) > End date ({__class__.colorMessage( strdate_end, 'red' )}</b>)"
            return v_return

        if len( collections ) == 0:
            v_return['htmlMessage'] = __class__.colorMessage( 'Select a collection!', 'red' )
            return v_return

        time_of_interest = f"{strdate_ini}/{strdate_end}"
        scenes = searchScenes( time_of_interest )
        total = len( scenes )
        if total == 0:
            v_return['htmlMessage'] = f"{__class__.colorMessage('No scenes: ', 'red' )} Dates ({time_of_interest})"
            return v_return

        v_return['htmlMessage'] =  __class__.colorMessage( f"Search: {total} scenes ({time_of_interest})", '#2E9AFE' )
        v_return['scenes'] = scenes

        return v_return

    @staticmethod
    def url(item):
        """
        Args:
            - item: Item collection
        Return: url (Tile XYZ)
        """
        r = requests.get( item.assets["tilejson"].href )
        r_json = r.json()
        url = r_json["tiles"][0]
        r.close()
        return url

    @staticmethod
    def toDates(scenes):
        """
        Args:
            - scenes: { scene: Item collection }
        Return: { 'date' = 'YYYY-MM-DD', 'items' = Item collection }
        """
        dateItem = lambda scenes, scene: scenes[ scene ].datetime.strftime( '%Y-%m-%d' )

        dates = [ dateItem( scenes, scene ) for scene in scenes ]
        dates = list( set( dates ) )

        scenes_date = {}
        for scene in scenes:
            date = dateItem( scenes, scene )
            item = scenes[ scene ]
            if not date in scenes_date:
                scenes_date[ date ] = [ item ]
                continue
            scenes_date[ date ].append( item )

        # Sort
        keys = list( scenes_date.keys() )
        keys.sort()
        return { k: scenes_date[k] for k in keys }

    @staticmethod
    def error(scenes, funcFinished, funcProgress, funcCancel):
        """
        Args:
            - scenes: { scene: Item collection }
            - funcProgress: params = ( label, count ), return None
            - funcCancel: params = None, return True/False
        Return: { scene: { url, status_code} } 
        """
        zoom_tile = 8
        scenes_error = {}
        count = 0
        for scene, item in scenes.items():
            count += 1

            funcProgress( scene, count )
            if funcCancel():
                scenes_error = {}
                break

            west, south, east, north = item.bbox
            part_lat = ( north - south ) / 3
            part_lng = ( east - west ) / 3

            west  += part_lat
            south += part_lng
            east  -= part_lat
            north -= part_lng

            tile = mercantile.tiles( west, south, east, north, zooms=[ zoom_tile ]).__next__()
            url = UtilScenes.url( item ).replace('{z}/{x}/{y}', f"{tile.z}/{tile.x}/{tile.y}")
            status_code = requests.get( url ).status_code
            if status_code != 200:
                scenes_error[ scene ] = { 'status_code': status_code, 'url': url, 'geometry': item.geometry }

        if len( scenes_error ) == 0:
            scenes_error = None

        funcFinished( scenes_error )

    @staticmethod
    def valid(scenes, scenes_error):
        """
        Args:
            - scenes: { scene: Item collection }
            - scenes_error: { scene: Item collection }
        Return: { scene: Item collection }
        """
        if scenes_error is None:
            return scenes

        return { scene: item for scene, item in scenes.items() if not scene in scenes_error }

    @staticmethod
    def footprint(scenes, ids=[]):
        """
        Args:
            - scenes: { scene: Item collection }
            - ids: List of scene (key)
        Return: geojson
        """
        def feature(scene):
            prop = { 'scene': scene }
            geom = scenes[ scene ].geometry
            return geojson.Feature( geometry=geom, properties=prop )

        if len( ids ) > 0:
            features = [ feature( item ) for item in ids ]
        else:
            features = [ feature( item ) for item in scenes ]

        return {
            'name': 'Scenes footprint',
            'type': 'FeatureCollection',
            'features': features
        }

    @staticmethod
    def footprintError(scenes_error, ids=[]):
        """
        Args:
            - scenes_error: { scene: {status_code, url, geometry } }
            - ids: List of scene (key)
        Return: GeoJSON
        """
        def feature(scene):
            prop = { 'scene': scene }
            geom = scenes_error[ scene ]['geometry']
            return geojson.Feature( geometry=geom, properties=prop )

        if len( ids ) > 0:
            features = [ feature( item ) for item in ids ]
        else:
            features = [ feature( item ) for item in scenes_error ]

        return {
            'name': 'Error scenes',
            'type': 'FeatureCollection',
            'features': features
        }


# GeojsonPopup

In [ ]:
class GeojsonPopup():
    def __init__(self, map_):
        self.map = map_
        self.layer = None
        self.popup = None
        
        self.style = { 'color': 'gray','opacity': 1, 'dashArray': '0', 'fillOpacity': 0.1, 'weight': 1 }
        self.hover_style = { 'color': 'white', 'dashArray': '0', 'fillOpacity': 0.5 }
        
    def load(self, geojson):
        if self.layer in self.map.layers:
            self.map.remove( self.layer )
            
        args = {
            'name': geojson['name'],
            'data': { k: geojson[ k ] for k in ( 'type', 'features') },
            'style': self.style,
            'hover_style': self.hover_style
        }
        self.layer = GeoJSON( **args )
        self.layer.on_click( self.on_click_layer )

    def zoom(self, feature_id):
        features = self.layer.data['features']
        if feature_id > len( features ) - 1:
            raise Exception(f"Error: Invalid Index ({feature_id})")
        
        feat = shape( features[ feature_id ]['geometry'] )
        ( minx, miny, maxx, maxy ) = scale( feat, 2, 2 ).bounds
        
        self.map.fit_bounds( [ (miny, minx), (maxy, maxx) ] )
        
    def addMap(self):
        self.map.add( self.layer )
        
    def on_click_layer(self, **kargs):
        """
        The 'kargs' can be differents arguments (ex. ID), depend of Geojson (layer)
        """
        def widgetTable():
            def htmlRows(values, type_row):
                html = ' '.join( [ f'<{type_row} id="#ELEMENT#_row">{item}</{type_row}>' for item in values ] )
                return f'<tr>{html}</tr>'.replace('#ELEMENT#', id_element )
        
            id_element = 'geojson_popup'#  
            html = """
                <style>
                #{ELEMENT}_row {
                  color: #BB9BF6;
                  border-style:solid;
                  border-color: #B6B5B8;
                }
                </style>
            """.replace('{ELEMENT}', id_element)
            html += '<table id="{ELEMENT}_table">'
            html += htmlRows( ['Field', 'Value' ],'th' )
            for key, value in properties.items():
                if key == 'style':
                    continue
                html += htmlRows( [ key, value ],'td' )
            html += '</table>'
            
            return HTML( html )
        
        feature = kargs['feature']
        properties = kargs['properties']
        center = shape( feature['geometry'] ).centroid
        if not self.popup in self.map.layers:
            self.popup = Popup(
                name=f"Popup - {self.layer.name}",
                location=(center.y, center.x),
                child=widgetTable(),
            )
            self.map.add( self.popup )
        else:
            self.popup.location=(center.y, center.x)
            self.popup.child=widgetTable()

        self.popup.open_popup()

    def clear(self):
        if self.layer in self.map.layers:
            self.map.remove( self.layer )
        self.map.remove( self.popup )


# ProviderWidgetProgress

In [6]:
class ProviderWidgetProgress():
    def __init__(self):
        self._cancel = False

        self.w_progress = IntProgress( min=0)
        self.w_item = Label()
        self.w_cancel = Button(
            description='Cancel',
            disabled=False,
            button_style='',
            icon='ban'
        )
        self.w_cancel.on_click( self._cancelTrue )
        
    def _cancelTrue(self, btn):
        self._cancel = True
    
    def init(self, total):
        self._cancel = False
        self.w_progress.max = total
        
    def widget(self):
        return HBox( [ self.w_progress, self.w_item , self.w_cancel ] )

    def start(self, callback, callback_data):
        def run():
            def progress(item, count):
                self.w_progress.value = count
                self.w_progress.description = f"{count} / {self.w_progress.max} :"
                self.w_item.value = item

            callback( *callback_data, progress, lambda : self._cancel )

        thread = threading.Thread(target=run)
        thread.start()
    
    def cancel(self):
        return self._cancel

    def progress(self, item, count):
        self.w_progress.value = count
        self.w_progress.description = f"{count} / {self.w_progress.max} :"
        self.w_item.value = item

# ProviderWidgetUploadGeojson

In [1]:
class ProviderWidgetUploadGeojson():
    def __init__(self, map_, feature_index_zoom=-1):
        self.map = map_
        self.feature_index_zoom = feature_index_zoom
        self.geojson_popup = GeojsonPopup( map_ )
        
        self.w_upload = FileUpload( accept='.geojson', multiple=False )
        self.w_upload.observe( self.on_observe_upload, names='value')

    def on_observe_upload(self, change ):
        if change.new is None:
            return

        # First key = Name of file
        name = next( iter( change.new.keys() ) )
        # Key 'content' = Binary string with data
        data = json.loads( change.new[ name ]['content'] )
        geojson = { 'name': name, 'type': data['type'], 'features': data['features'] }
        
        if self.feature_index_zoom > -1:
            if self.feature_index_zoom > len( data['features'] ) - 1:
                raise Exception(f"Error: Invalid Index ({self.feature_index_zoom})")

            geojson['features'] = [ data['features'][ self.feature_index_zoom ] ]
            self.geojson_popup.load( geojson )
            self.geojson_popup.zoom( self.feature_index_zoom )
        else:
            self.geojson_popup.load( geojson )

        self.geojson_popup.addMap()
        
    def widget(self):
        return self.w_upload
    
    def clear(self):
        self.geojson_popup.clear()


# ProviderDrawControlRectangle

In [6]:
class ProviderDrawControlRectangle():
    def __init__(self, map_, position):
        def createControl():
            control = DrawControl(position=position)
            control.edit, control.remove = False, False
            control.circlemarker = {}
            control.polyline = {}
            control.polygon = {}
            control.rectangle = {
                "shapeOptions": {
                    "fillColor": "#fca45d",
                    "color": "#fca45d",
                    "fillOpacity": 0.5
                }
            }
            control.on_draw( self.on_draw )
            return control
        
        self.geometry = None
        map_.add( createControl() )
        
    def on_draw(self, target, action, geo_json):
        def changeStyle():
            style =  geo_json['properties']['style']
            style['color'] = 'blue'
            style['fill'] = False
            # Update target
            target.data = [ geo_json ]

        if not action == 'created':
            return

        self.geometry = geo_json['geometry']
        changeStyle()

# ProviderLabelCoordinateControl

In [6]:
class ProviderLabelCoordinateControl():
    def __init__(self, map_, position):
        self.label = Label()
        self.f_coord = lambda c: f"{round( c , 4 )} º"
        
        control = WidgetControl( widget=self.label, position=position)
        map_.add( control )
        map_.on_interaction( self.on_interaction )
        
    def on_interaction(self, **kwargs):
        if kwargs['type'] == 'mousemove':
            f_coord = lambda c: f"{round( c , 4 )} º"
            [ y, x ] = kwargs['coordinates']
            self.label.value = f"{self.f_coord(x)}, {self.f_coord(x)}"

# ProviderWidgetSearchScenes

In [6]:
class ProviderWidgetSearchScenes():
    def __init__(self, controlRectangle, funcProcessScenes):
        self.controlRectangle = controlRectangle
        self.funcProcessScenes = funcProcessScenes

        self.w_date_end = DatePicker(description='End date')
        self.w_date_end.value = datetime.today().date()

        self.w_date_ini = DatePicker(description='Initial date')
        self.w_date_ini.value = self.w_date_end.value - timedelta(days=30)

        collections = [ 'landsat-c2-l2', 'sentinel-2-l2a']
        self.w_collections = SelectMultiple(
            options=collections,
            description='Collections',
            disabled=False
        )
        self.w_collections.value = [ collections[0] ]
        
        self.w_button = Button(
            description='Search scenes',
            disabled=False,
            button_style='',
            icon='satellite-dish'
        )
        self.w_button.on_click( self._on_click )
        
        self.w_html_result = HTML()
        
    def _on_click(self, btn):
        btn.disabled = True
        scenes = None
        
        if self.controlRectangle.geometry is None:
            html = UtilScenes.colorMessage( f"Error: Create Rectangle on Map", 'red' )
        else:
            args = {
                'collections': self.w_collections.value,
                'date_ini': self.w_date_ini.value,
                'date_end': self.w_date_end.value,
                'geom_json': self.controlRectangle.geometry
            }
            r = UtilScenes.search( **args )
            html = r['htmlMessage']
            scenes = r['scenes']
            
        self.w_html_result.value = html
            
        btn.disabled = False

        self.funcProcessScenes( scenes )

    def widget(self):
        return {
            'search': HBox( [ VBox( [ self.w_date_ini, self.w_date_end ] ), self.w_collections, self.w_button ] ),
            'result': self.w_html_result
        }

# ProviderWidgetProcessScenes

In [6]:
class ProviderWidgetProcessScenes():
    def __init__(self, map_, position_date, position_scene_error, w_output):
        self.map = map_
        self.w_output = w_output
        self.position_date = position_date
        self.scenes_date, self.scenes_error = None, None
        self.layer_scenes_error = GeojsonPopup( map_ )
        self.layers_scenes_date = []

        self.w_button_error = Button(
            description='Show error scenes',
            button_style='',
            icon='images'
        )
        self.w_button_error.on_click( self.on_click_error )
        
        self.w_button_clear = Button(
            description='Remove error',
            button_style='',
            icon='broom'
        )
        self.w_button_clear.on_click( self.on_click_clear )
        
        # html_valid Hide/Show
        self.w_html_valid = HTML()
        
        # Widgets for controls
        #.)  Control date, add top left Hide/Show
        self.w_label_dates = Label('')
        self.w_selection_date = SelectionSlider(
            options=[''],
            value='',
            description='Scenes:',
        )
        self.w_selection_date.observe( self.on_observe_selection_date, names='value')
        self.w_label_date_position = Label('')
        self.control_date = None # Create in run():finish()
        # Hide
        widgets = ( self.w_button_error, self.w_button_clear, self.w_html_valid )
        self._toggleLayoutDisplay( widgets, False )
        
    def _toggleLayoutDisplay(self, widget, is_visible):
        # Values for display: block = Visible, none = Hide
        lyt_display = 'block' if is_visible else 'none'
        
        if type( widget ) in ( tuple, list ):
            for w in widget:
                w.layout.display = lyt_display
            return

        widget.layout.display = lyt_display

    def _removeLayersSceneDate(self):
        for layer in self.layers_scenes_date:
            if layer in self.map.layers:
                self.map.remove( layer )
        self.layers_scenes_date.clear()
    
    def _addControlDate(self):
        w = HBox( [ self.w_label_dates, self.w_selection_date, self.w_label_date_position ] )
        self.control_date = WidgetControl( widget=w, position=self.position_date )
        self.map.controls = ( self.control_date, *self.map.controls ) # Add top control_date
    
    def run(self, scenes):
        def finished(scenes_error):
            widgetProgress.close()
            if progress.cancel():
                widgetMessage.value = UtilScenes.colorMessage( 'Canceled by user', 'red' )
                return

            existsError = not scenes_error is None
            
            # Date scenes
            scenes_valid = UtilScenes.valid( scenes, scenes_error ) if existsError else scenes
            self.scenes_date = UtilScenes.toDates( scenes_valid )
            options = [ scene for scene in self.scenes_date ]
            self.w_label_dates.value = f"{options[0]}/{options[-1]} ({len(options)} dates)"
            self.w_selection_date.options = options
            self.w_selection_date.value = options[0]
            self.on_observe_selection_date( { 'new': options[0] } )
            self._addControlDate()
            
            self.w_html_valid.value = UtilScenes.colorMessage( f"Valid: {len(scenes_valid)} scenes", '#2E9AFE' )
            
            self._toggleLayoutDisplay( self.w_html_valid , True )

            if not existsError:
                with self.w_output:
                    widgetMessage.value = UtilScenes.colorMessage( 'All scenes are OK', '#2E9AFE' )
                return
            
            with self.w_output:
                widgetMessage.value = UtilScenes.colorMessage( f"{len( scenes_error)} scenes error", 'red' )
            
            self.scenes_error = scenes_error
            self.layer_scenes_error.load( UtilScenes.footprintError( scenes_error ) )
            
            self._toggleLayoutDisplay( self.w_button_error, True )

        # Hide
        widgets = ( self.w_button_error, self.w_button_clear, self.w_html_valid )
        self._toggleLayoutDisplay( widgets, False )
        if self.control_date in self.map.controls:
            self.map.remove( self.control_date )

        self.w_output.clear_output()
        
        # Clear
        self.scenes_date, self.scenes_error = None, None
        self.layer_scenes_error.clear()
        self._removeLayersSceneDate()
        
        if scenes is None:
            return
        
        # Calculate "scenes_error"
        progress = ProviderWidgetProgress()
        progress.init( len( scenes ) )
        
        widgetProgress = progress.widget()
        widgetMessage = HTML()
        with self.w_output:
            display( VBox( [ widgetProgress, widgetMessage ] ) )
        
        progress.start( UtilScenes.error, ( scenes, finished ) )

    def on_click_error(self, btn):
        def tableHtml():
            def htmlRows(values, type_row):
                html = ' '.join( [ f'<{type_row} id="#ELEMENT#_row">{item}</{type_row}>' for item in values ] )
                return f'<tr>{html}</tr>'.replace('#ELEMENT#', id_element )

            btn.disabled = True

            id_element = 'scenes_error'
            html = """
                <style>
                #{ELEMENT}_row {
                  color: #D6C9B6;
                  border-style:solid;
                  border-color: #B6B5B8;
                }
                #{ELEMENT}_a:link {
                  color: green;
                  background-color: transparent;
                  text-decoration: none;
                }
                #{ELEMENT}_a:visited {
                  color: pink;
                  background-color: transparent;
                  text-decoration: none;
                }
                #{ELEMENT}_a:hover {
                  color: red;
                  background-color: transparent;
                  text-decoration: underline;
                }
                #{ELEMENT}_a:active {
                  color: yellow;
                  background-color: transparent;
                  text-decoration: underline;
                }
                </style>
            """.replace('{ELEMENT}', id_element)

            html += '<table>'
            html += htmlRows( [ f"{len( self.scenes_error )} erros" ],'th' )
            html += htmlRows( ['Id', 'Code' ],'th' )
            for key, values in self.scenes_error.items():
                html_link = f"""<a id="#ELEMENT#_a" href="{values['url']}" target="_blank">{key}</a>""".replace('#ELEMENT#', id_element )
                html += htmlRows( [ html_link, values['status_code'] ],'td' )
            html += '</table>'
            
            return HTML( html )

        btn.disabled = False
        
        self.w_output.clear_output()
        with self.w_output:
            display( tableHtml() )
        
        self.layer_scenes_error.addMap()
        
        self._toggleLayoutDisplay( self.w_button_clear, True )
        
        btn.disabled = False
        
    def on_click_clear(self, btn):
        btn.disabled = True
        
        self._toggleLayoutDisplay( btn, False )
        
        self.w_output.clear_output()
        self.layer_scenes_error.clear()
        
        btn.disabled = False
        
    def on_observe_selection_date(self, change):
        # Clean last layers
        self._removeLayersSceneDate()

        # Position
        idx = list( self.scenes_date.keys() ).index( change['new'] )
        self.w_label_date_position.value = f"({idx+1}º)"
        
        names = []
        for item in self.scenes_date[ change['new'] ]:
            layer = TileLayer(name=item.id, url=UtilScenes.url( item ), attribution='Microsoft Planetary Computer')
            self.layers_scenes_date.append( layer )
            self.map.add( layer )
            names.append( item.id )
            
        msg = '\n'.join( names )
        self.w_selection_date.description_tooltip = f"{msg}"

    def widgets(self):
        return {
            'button_error': self.w_button_error,
            'button_clear': self.w_button_clear,
            'html_valid': self.w_html_valid, 
        }

# Provider Magnifying Glass Layer
    - Issue "Update magnifying image when back this tools"
        - Date: 2023-01-16
        - Local: ProviderWidgetMagnifyingGlassLayer.on_interaction_map.refresh  
        - Status: Need study how argumetn for "send"    

In [11]:
class ProviderWidgetMagnifyingGlassLayer():
    def __init__(self, map_, position='bottomleft'):
        self.map = map_

        self.layer = None
        
        self.no_source = '-- None --'
        self.sources = {
            self.no_source: None,
            'Esri': basemap_to_tiles( basemaps.Esri.WorldImagery ),
            'Google': TileLayer( url='http://mt0.google.com/vt/lyrs=s&x={x}&y={y}&z={z}', attribution='Google')
        }
        self.w_sources = Dropdown(
            options=list( self.sources.keys() ),
            value=self.no_source,
            description='Magnifying:',
            description_tooltip='Select source for Magnifying Layer',
            icon='check'
        )
        self.w_sources.observe( self.on_observe_source, names='value')
        
        self.w_zoom = IntSlider(
            value=0,
            min=0, max=6,
            step=1,
            description="Offset Zoom:",
            description_tooltip='Multipliy the current zoom for Magnifying',
            disabled=True
        )
        self.w_zoom.observe( self.on_observe_zoom, names='value')

        self.w_ = VBox( [ self.w_sources, self.w_zoom ] )
        self.map.add( WidgetControl( widget=self.w_, position='bottomleft' ) )
        
        self.map.on_interaction(self.on_interaction_map)
        
    def _addLayer(self, source, zoom):
        self.layer = MagnifyingGlass( layers=[ self.sources[ source ] ], zoom_offset=zoom )
        self.layer.name = source
        self.map.add( self.layer )

    def on_interaction_map(self, **kwargs):
        def refresh(): # ISSUE
            y, x = self.map.center
            content = {
                'type': 'mouseover',
                'coordinates': [ y+0.001, x+0.001 ]
            }
            test = {
                'comm_id': 'u-u-i-d',
                'data': {
                    'method': 'custom',
                    'content': content,
                    'buffers': None
                }
            }
            self.layer.send( **test )
            print(1)
            
        if not kwargs.get('type') == 'click' or self.w_sources.value == self.no_source:
            return

        # Return Magnifying
        if self.w_sources.disabled:
            self.w_sources.disabled = False
            self.layer.zoom_offset = self.w_zoom.value
            self.map.add( self.layer )
            #refresh() # ISSUE
            return
        
        self.w_sources.disabled = True
        self.map.remove( self.layer )
        
    def on_observe_source(self, change):
        if self.layer in self.map.layers:
            self.map.remove( self.layer )
        
        if self.no_source == change['new']:
            self.layer = None
            self.w_zoom.disabled = True
            return

        self._addLayer( change['new'], self.w_zoom.value )
        self.w_zoom.disabled = False
                       
    def on_observe_zoom(self, change):
        if self.no_source == self.w_sources.value or self.w_sources.disabled:
            return
        
        if self.layer in self.map.layers:
            self.map.remove( self.layer )
        self._addLayer( self.w_sources.value, change['new'] )
